<h1>Newton-Raphson<h1> 
<p>A method for a system of n nonlinear equations with n unknowns.</p>

In [37]:
import torch as th
import matplotlib.pyplot as plt

In [39]:
class NewtonRaphson:

  def __init__(self, x, tol, functions):
    self.x = x
    self.tolerance = tol
    self.functions = functions

  # Calculate the jacobian matrix
  def jacobianMatrix(self):
    jacobian = []
    for i in range(len(self.functions)):
      jacobian.append(list(map(lambda x: x.item(), list(th.autograd.functional.jacobian(self.functions[i], self.x)))))
    return th.tensor(jacobian)
  
  # Calculate inverse jacobian
  def inverseJacobian(self):
    return th.inverse(self.jacobianMatrix())

  # Calculate the function by replacing the unknowns variables with their values
  def functionCalculation(self):
    functionValues = []
    for function in self.functions:
      functionValues.append(function(self.x))
    return th.tensor([functionValues])

  # Main Newton-Raphson algorithm
  def newtonRaphson(self):
    X = []
    localErrors = [[self.x[0].item(), self.x[1].item()]]
    counter = 0
    while counter < 10 and (localErrors[counter][0] >= self.tolerance and localErrors[counter][1] >= self.tolerance):

      # Calculate the jacobian and find anew solution given previous solution
      X.append(th.sub(self.x, th.transpose(th.mm(self.inverseJacobian(), th.transpose(self.functionCalculation(), 0, 1)), 0, 1)))

      # Calculate the local error for x and y variables 
      localErrors.append([])
      localErrors[counter+1].append(abs(self.x[0].item() - X[counter][0][0].item()))
      localErrors[counter+1].append(abs(self.x[1].item() - X[counter][0][1].item()))
      

      # Update the solution x
      self.x = th.tensor(list(map(lambda x: x.item(), list(X[counter][0]))))

      # Update the counter
      counter += 1

    return X, localErrors

  # Graph the error for the x and y variables
  def graphError(self, errors):
    xError = [x[0] for x in errors]
    yError = [y[1] for y in errors]
    fig, axs = plt.subplots(1, 2)
    axs[0].plot(xError)
    axs[0].set_title("X Variable Error")
    axs[1].plot(yError)
    axs[1].set_title("Y Variable Error")
    fig.tight_layout()



In [ ]:
# Initial X
initialX = th.tensor([1.98, 1.02])

functions = []
functions.append(lambda x: x[0]+x[0]*x[1]-4)
functions.append(lambda x: x[0]+x[1]-3)
tolerance = 0.0003

NR = NewtonRaphson(initialX, tolerance, functions)
x, errors = NR.newtonRaphson()

# Show X results
for iteration in x:
  print(iteration[0])

# Show x and y error results
for iteration in errors:
  print(iteration)

# Graph the error
NR.graphError(errors)
